# Basis size using Gausslets for arbitrary molecules

In this notebook we want to estimate the size of the basis, $N$ we want to use in the phase estimation protocol of article "Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity" (https://journals.aps.org/prx/pdf/10.1103/PhysRevX.8.041015) using Gausslets. Notice that the T Complexity of the algorithm will be 
$$\frac{24\sqrt{2}\pi \lambda}{\Delta E}N,$$
where $$\lambda = O(N^2)$$ is the sum of the absolute values of the coefficients in the Hamiltonian. However, this basis cannot be the usual gaussian but rather needs to be the gausslet-based, for instance. Here we calculate the size of the basis $N$ for an arbitary molecule from psi4. We will follow appendix A from "Supplemental Material for Multi-sliced Gausslet Basis Sets for Electronic Structure" (https://arxiv.org/pdf/1809.10258.pdf).

In [1]:
from scipy import integrate
import numpy as np
import scipy.optimize

Let us suppose we have a list of atoms, saved at atom_list, and each atom having characteristics x, y and z. The first thing we need is to set the hyperparameters. Those are $b$ and $d$; and $s$ and $a$. We will use a similar choice as those in the article. However, we should be aware of a unit change: the Gausslet article works in atomic units, whereas psi4 works in amstrongs.

In [2]:
b = 13
d = 9

a = (.5) #**(1/3)
s = (.5) #**(1/3)

# 1 bohr = 0.529177210903 amstrongs
bohr_to_amstrong = 0.529177210903

In [3]:
class atom():
    def __init__(self,x,y,z):
        self.x = x
        self.y = y
        self.z = z

# Substitute this list of atoms by the ones read out from our molecule.
H1 = atom(0,0,0)
H2 = atom(1,0,0)
H3 = atom(0,1,0)
H4 = atom(0,0,1)
atoms = [H1, H2, H3, H4]

Now we have to define the function indicating the density gausslets. That is
$$\rho(\vec{x}) = \left[ \sum_\alpha \rho_\alpha(\vec{x})\right]^{1/2} + \frac{1}{d},$$ 
where $\alpha$ represent the different atoms. On the other hand
$$\rho_{\alpha,z}(\vec{x}= (i,j,k)) = \frac{1}{s\sqrt{a_{\alpha}^2 + (k-\alpha.z)^2 }},$$
where $\alpha.z$ represents the coordinate $z$ of atom $\alpha$, and similarly for $x$ and $y$.
In two dimension that changes to 
$$\rho_{\alpha,y}(\vec{x}= (i,j,k)) = \frac{1}{s\sqrt{a_{\alpha}^2 + (j-\alpha.y)^2 + d_{z= k}^2 }}; \qquad d_{z=k} = \min_\alpha |k - \alpha.z|.$$
And for three dimensions
$$\rho_{\alpha,x}(\vec{x}= (i,j,k)) = \frac{1}{s\sqrt{a_{\alpha}^2 + (i-\alpha.x)^2 + d_{z= k, y= j}^2 }}; \qquad d_{z= k, y= j} = \min_\alpha |(j,k) - (\alpha.y,\alpha.z)|.$$

Finally we need to fit the $a_\alpha$ such that the density at the atoms is $(as)^{-1}$, using the first and second equations. Finally
$$\rho_{\alpha}(\vec{x}= (i,j,k)) = \rho_{\alpha,z}\rho_{\alpha,y}\rho_{\alpha,x}.$$

However, I feel that instead of $a_\alpha$ and $s$ in the above formulas, one should attempt to have the cubic root such that 3d does not spoil it. Also, one would need to perform the fit of $a_\alpha$ probably using the final (product of densities) function.

In [4]:
def rho_z_atom(x,y,z, atom, atoms, a = a, s = s):
    density = 1/(s*np.sqrt(a**2 + (z-atom.z)**2))
    return density

def rho_y_atom(x,y,z, atom, atoms, a = a, s = s):
    d_list = []
    for at in atoms:
        d_list += [np.abs(z-at.z)]
    d = min(d_list)
    
    density = 1/(s*np.sqrt(a**2 + (y-atom.y)**2 + d**2))
    return density

def rho_x_atom(x,y,z, atom, atoms, a = a, s = s):
    d_list = []
    position = np.array([y,z])
    for at in atoms:
        at_position = np.array([at.y,at.z])
        d_list += [np.linalg.norm(position - at.position)]
    d = min(d_list)
    
    density = 1/(s*np.sqrt(a**2 + (x-atom.x)**2 + d**2))
    return density

In [7]:
def density(x,y,z, atoms, a_alpha_list = None ,a = a, s = s):
    # First let us check that we are not too far away from any atom:
    d_list = []
    position = np.array([x,y,z])
    for at in atoms:
        at_position = np.array([at.x, at.y,at.z])
        d_list += [np.linalg.norm(position - at_position)]
    
    d = min(d_list)
    if d > b*bohr_to_amstrong:
        return 0
    
    
    if a_alpha_list == None:
        dens = 0
        for atom in atoms:
            dens += rho_z_atom(x,y,z,atom, atoms)*rho_z_atom(x,y,z,atom, atoms)*rho_z_atom(x,y,z,atom, atoms)
            return np.sqrt(dens) +1/(d*bohr_to_amstrong)

    else:
        for (atom, a_alpha) in zip(atoms, a_alpha_list):
            dense = 0
            dens += rho_z_atom(x,y,z,atom, atoms, a = a_alpha)*rho_z_atom(x,y,z,atom, atoms,a = a_alpha)*rho_z_atom(x,y,z,atom, atoms, a = a_alpha)

            return np.sqrt(dens) + 1/(d*bohr_to_amstrong)

Now we are in condition to integrate over the total density, and then we need to fit the $a_\alpha$ such that the final density in the atomic nuclei is fixed at $(as)^{-3}$. Notice that the 3 appears because we are integrating in 3 dimensions.

In [8]:
# First calculate the integration limit
x_list = []
y_list = []
z_list = []
for atom in atoms:
    x_list.append(atom.x)
    y_list.append(atom.y)
    z_list.append(atom.z)
    
upper_x_lim = max(x_list)+ b*bohr_to_amstrong
upper_y_lim = max(y_list)+ b*bohr_to_amstrong
upper_z_lim = max(z_list)+ b*bohr_to_amstrong

lower_x_lim = min(x_list)- b*bohr_to_amstrong
lower_y_lim = min(y_list)- b*bohr_to_amstrong
lower_z_lim = min(z_list)- b*bohr_to_amstrong

def density_func(z,y,x, a_alpha_list = None):

    dens = density(x,y,z, atoms, a_alpha_list ,a = a, s = s)

    return dens

# TODO: integrate using this function
(N, N_err)  = integrate.tplquad(density_func, 
                                lower_x_lim, upper_x_lim, 
                                lambda z: lower_y_lim, lambda z: upper_y_lim, 
                                lambda z, y: lower_z_lim, lambda z, y: upper_z_lim, 
                                epsabs = 1e-1, epsrel = 1e-1)
print(N, N_err)

3425.6352433662164 200.46619880252416


In [ ]:
def F(a_alpha_list = [a]*len(atoms)):
    result = []
    for (atom, a_alpha) in zip(atoms, a_alpha_list):
        z = float(atom.z)
        y = float(atom.y)
        x = float(atom.x)
        print('x,y,z',x,y,z)
        print(a_alpha_list)

        dens = density_func(atom.z, atom.y, atom.x, list(a_alpha_list))
        print('density d',d)
        result.append(dens - (a*s)**(-3)-1/(d*bohr_to_amstrong))
    
    return result

print('a, a, a',[a]*len(atoms))

# TODO: I do not know how to indicate maximum and minimum values for the a_alphas... The following line fails
a_alpha_list = scipy.optimize.broyden2(F, [a]*len(atoms), f_tol=1e-1)

print(a_alpha_list)

In [ ]:
lista = [0]*3
print(lista)

Let us now calculate the final cost. Assuming that we want a precision $\Delta E$ such that $\lambda / \Delta E = 10^3 N^2$ we have the following

In [14]:
lam_delta = 1e3 # NOTICE: This number has to be checked!

cost = lam_delta * N**3 *24 *np.sqrt(2)* np.pi
t = cost * 100e-9
cost = format(cost , 'e')
t = format(t , 'e')
print('Number of T gates:', cost)
print('Serial running time (s) assuming 100ns per T gate', t)
# Un mes son 2.628e+6 segundos


Number of T gates: 4.286467e+15
Serial running time (s) assuming 100ns per T gate 4.286467e+08
